In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import cv2
import torchvision.transforms as transforms


In [ ]:
class image_dataset(Dataset):
    def __init__(self, directory, transform=None):
        self.directory=directory
        self.transform = transform
        data = []
        classes=os.listdir(directory)
        for i in range(len(classes)):
            files=os.listdir(directory+'/'+classes[i])
            for file_name in files:
               temp_data=cv2.imread(directory+"/"+classes[i]+"/"+file_name)
               #temp_data=cv2.resize(temp_data,(150,150))
               temp_data2=np.array([temp_data,i],dtype=object)
               data.append(temp_data2)
        self.data =np.asarray(data)



    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        input = self.data[index][0]
        target = self.data[index][1]
        #input = torch.tensor(input,dtype=torch.float32)
        #input=torch.permute(input,(2,0,1))
        target = torch.tensor(target)
        if(self.transform!=None):
          input=self.transform(input)
        return (input, target)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    #transforms.ToTensor(),
    #transforms.permute((2,0,1))
])

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
print(model)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
model.classifier[6]=nn.Linear(4096,6)

In [ ]:
path_train="/content/drive/MyDrive/Image_class_dataset/seg_train/seg_train"
path_val="/content/drive/MyDrive/Image_class_dataset/seg_val"

train_set=image_dataset(path_train,preprocess)
val_set=image_dataset(path_val,preprocess)

In [ ]:
batch_size = 128
shuffle = True
pin_memory = True
num_workers = 2
num_epochs= 100
train_loader = DataLoader(dataset=train_set, shuffle=shuffle, batch_size=batch_size,num_workers=num_workers,pin_memory=pin_memory)
valid_loader=DataLoader(dataset=val_set, shuffle=shuffle, batch_size=batch_size,num_workers=num_workers,pin_memory=pin_memory)
#model=NaturalSceneClassification()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [ ]:
device = 'cpu'
for epoch in range(1, num_epochs + 1):
    model.train()
    train_loss = 0.0
    for data, target in train_loader:
        # print('input :',data)
        # print('target :',target)
        # print(data.size(0))
        data = data.to(device)
        target = target.to(device)
        # clear-the-gradients-of-all-optimized-variables
        optimizer.zero_grad()
        # forward-pass: compute-predicted-outputs-by-passing-inputs-to-the-model

        output = model(data)
        # calculate-the-batch-loss
        loss = criterion(output, target)
        # backward-pass: compute-gradient-of-the-loss-wrt-model-parameters
        loss.backward()
        # perform-a-single-optimization-step (parameter-update)
        optimizer.step()
        # print(data)
        # print(target)
        # update-training-loss
        # print(loss.item())
        # print(len(train_loader.sampler))
        train_loss += loss.item() * data.size(0)
    model.eval()
    val_loss=0.0
    for data, target in valid_loader:
        data=data.to(device)
        target=target.to(device)
        pred = model(data)
        loss1=criterion(pred,target)
        val_loss+=loss1.item()*data.size(0)

    val_loss=val_loss/len(valid_loader.sampler)
    train_loss = train_loss / len(train_loader.sampler)
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss,val_loss))


Epoch: 1 	Training Loss: 35.362314 	Validation Loss: 25.686640
Epoch: 2 	Training Loss: 7.690667 	Validation Loss: 18.366919
Epoch: 3 	Training Loss: 3.635782 	Validation Loss: 12.065002
Epoch: 4 	Training Loss: 2.175373 	Validation Loss: 9.753644
Epoch: 5 	Training Loss: 1.619329 	Validation Loss: 9.252832
Epoch: 6 	Training Loss: 1.176578 	Validation Loss: 7.469759
Epoch: 7 	Training Loss: 1.110402 	Validation Loss: 6.703886
Epoch: 8 	Training Loss: 0.895999 	Validation Loss: 6.990613
Epoch: 9 	Training Loss: 0.728353 	Validation Loss: 8.013996
Epoch: 10 	Training Loss: 0.632132 	Validation Loss: 9.195711
Epoch: 11 	Training Loss: 0.603660 	Validation Loss: 9.238815
Epoch: 12 	Training Loss: 0.529293 	Validation Loss: 9.632605
Epoch: 13 	Training Loss: 0.476908 	Validation Loss: 10.677289
Epoch: 14 	Training Loss: 0.466652 	Validation Loss: 10.550591
Epoch: 15 	Training Loss: 0.411513 	Validation Loss: 10.364711
Epoch: 16 	Training Loss: 0.324630 	Validation Loss: 10.568663
Epoch: 17